In [1]:
import pandas as pd
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

from tsfresh import extract_features, select_features
from third_party.feature_selector import FeatureSelector

In [2]:
window_size = 60
n_features = 106
filename = 'data/trace_201708/server_usage.csv'

In [3]:
df = pd.read_csv(filename, header=None)
df = df[[0, 1, 2]]
df.rename(columns={
    0: 'time',
    1: 'machine_id',
    2: 'cpu'
}, inplace=True)

df['cpu'] = df['cpu'].fillna(0)
df.sort_values(['machine_id', 'time'], inplace=True)

In [4]:
df_grp = df.copy().set_index(['machine_id', 'time'], drop=True)
df_grp

cpu
machine_id time        
1          39600  44.62
           39900  35.60
           40200  30.98
           40500  29.52
           40800  30.18
...                 ...
1313       81300  13.76
           81600  14.72
           81900  13.42
           82200  13.56
           82500  14.32

[187963 rows x 1 columns]

In [5]:
df_rolled = roll_time_series(
    df,
    column_id='machine_id',
    column_sort='time',
    max_timeshift=window_size-1,
    min_timeshift=window_size-1,
)


/opt/conda/envs/py39/lib/python3.9/site-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Rolling: 100%|██████████| 72/72 [00:09<00:00,  7.26it/s]


In [6]:
df_rolled

,time,machine_id,cpu,id
0,39600,1,44.62,"(1, 57300)"
1,39900,1,35.60,"(1, 57300)"
2,40200,1,30.98,"(1, 57300)"
3,40500,1,29.52,"(1, 57300)"
4,40800,1,30.18,"(1, 57300)"
...,...,...,...,...
5068795,81300,1313,13.76,"(1313, 82500)"
5068796,81600,1313,14.72,"(1313, 82500)"
5068797,81900,1313,13.42,"(1313, 82500)"
5068798,82200,1313,13.56,"(1313, 82500)"


In [7]:
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters

extracted_features = extract_features(
    df_rolled.drop('machine_id', axis=1),
    column_id='id',
    column_sort='time',
    column_value='cpu',
    impute_function=impute,
    default_fc_parameters=EfficientFCParameters(),
) 

Feature Extraction: 100%|██████████| 120/120 [07:26<00:00,  3.72s/it]


In [8]:
extracted_features

cpu__variance_larger_than_standard_deviation  \
1    57300                                           1.0   
     57600                                           1.0   
     57900                                           1.0   
     58200                                           1.0   
     58500                                           1.0   
...                                                  ...   
1313 81300                                           1.0   
     81600                                           1.0   
     81900                                           1.0   
     82200                                           1.0   
     82500                                           1.0   

            cpu__has_duplicate_max  cpu__has_duplicate_min  \
1    57300                     0.0                     0.0   
     57600                     0.0                     0.0   
     57900                     0.0                     0.0   
     58200                     0.0                     0.0   
     58500                     0.0                     0.0   
...                            ...                     ...   
1313 81300                     0.0                     0.0   
     81600                     0.0                     0.0   
     81900                     0.0                     0.0   
     82200                     0.0                     0.0   
     82500                     0.0                     0.0   

            cpu__has_duplicate  cpu__sum_values  cpu__abs_energy  \
1    57300                 1.0      2075.200004     73049.099452   
     57600                 1.0      2061.060004     71987.185429   
     57900                 0.0      2055.940004     71648.855848   
     58200                 0.0      2053.760004     71518.535453   
     58500                 0.0      2049.680004     71294.298656   
...                        ...              ...              ...   
1313 81300                 0.0      1638.439997     47027.796582   
     81600                 0.0      1625.239998     46464.948601   
     81900                 0.0      1609.359998     45786.554998   
     82200                 0.0      1595.759998     45232.763007   
     82500                 0.0      1582.259998     44663.873000   

            cpu__mean_abs_change  cpu__mean_change  \
1    57300              4.749830         -0.223051   
     57600              4.613559         -0.086780   
     57900              4.535254         -0.008475   
     58200              4.538983         -0.012203   
     58500              4.584746         -0.080339   
...                          ...               ...   
1313 81300              3.217627         -0.240000   
     81600              3.210508         -0.247119   
     81900              3.196271         -0.232881   
     82200              3.187458         -0.241695   
     82500              3.135254         -0.293898   

            cpu__mean_second_derivative_central  cpu__median  ...  \
1    57300                             0.162414        34.38  ...   
     57600                             0.031379        34.20  ...   
     57900                             0.012586        33.92  ...   
     58200                            -0.020172        33.92  ...   
     58500                            -0.032586        33.92  ...   
...                                         ...          ...  ...   
1313 81300                            -0.026207        27.18  ...   
     81600                             0.026724        27.14  ...   
     81900                            -0.016897        26.72  ...   
     82200                            -0.031897        26.29  ...   
     82500                            -0.021724        26.15  ...   

            cpu__fourier_entropy__bins_5  cpu__fourier_entropy__bins_10  \
1    57300                      0.999869                       1.631021   
     57600                      1.131502                       1.678262   
     57900                

In [9]:
y = df.groupby('machine_id').apply(lambda x: x.set_index('time')['cpu'].shift(-1)[:-1])

In [10]:
y = y[y.index.isin(extracted_features.index)]
extracted_features = extracted_features[extracted_features.index.isin(y.index)]

In [11]:
y.shape, extracted_features.shape

((109363,), (109363, 777))

## Feature Selector

In [12]:
fs = FeatureSelector(data=extracted_features, labels=y)

In [13]:
fs.identify_single_unique()

42 features with a single unique value.



In [14]:
fs.identify_collinear(correlation_threshold=0.98)

70 features with a correlation magnitude greater than 0.98.



/Projects/adaptive_prediction_models/third_party/feature_selector.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_collinear = record_collinear.append(temp_df, ignore_index = True)
/Projects/adaptive_prediction_models/third_party/feature_selector.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_collinear = record_collinear.append(temp_df, ignore_index = True)
/Projects/adaptive_prediction_models/third_party/feature_selector.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_collinear = record_collinear.append(temp_df, ignore_index = True)
/Projects/adaptive_prediction_models/third_party/feature_selector.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [15]:
fs.identify_zero_importance(task='regression', eval_metric='l2', n_iterations=10, early_stopping=True)

Training Gradient Boosting Model



/opt/conda/envs/py39/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/envs/py39/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/envs/py39/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stoppin


347 features with zero or negative importance after one-hot encoding.



In [16]:
extracted_features = fs.remove(methods=['single_unique', 'collinear', 'zero_importance'])

Removed 405 features.


In [17]:
extracted_features

cpu__has_duplicate  cpu__sum_values  cpu__abs_energy  \
1    57300                 1.0      2075.200004     73049.099452   
     57600                 1.0      2061.060004     71987.185429   
     57900                 0.0      2055.940004     71648.855848   
     58200                 0.0      2053.760004     71518.535453   
     58500                 0.0      2049.680004     71294.298656   
...                        ...              ...              ...   
1313 81000                 0.0      1655.259997     47773.595376   
     81300                 0.0      1638.439997     47027.796582   
     81600                 0.0      1625.239998     46464.948601   
     81900                 0.0      1609.359998     45786.554998   
     82200                 0.0      1595.759998     45232.763007   

            cpu__mean_abs_change  cpu__mean_change  \
1    57300              4.749830         -0.223051   
     57600              4.613559         -0.086780   
     57900              4.535254         -0.008475   
     58200              4.538983         -0.012203   
     58500              4.584746         -0.080339   
...                          ...               ...   
1313 81000              3.234576         -0.256949   
     81300              3.217627         -0.240000   
     81600              3.210508         -0.247119   
     81900              3.196271         -0.232881   
     82200              3.187458         -0.241695   

            cpu__mean_second_derivative_central  cpu__standard_deviation  \
1    57300                             0.162414                 4.609498   
     57600                             0.031379                 4.449182   
     57900                             0.012586                 4.473442   
     58200                            -0.020172                 4.508694   
     58500                            -0.032586                 4.608862   
...                                         ...                      ...   
1313 81000                            -0.009828                 5.928511   
     81300                            -0.026207                 6.173019   
     81600                             0.026724                 6.379043   
     81900                            -0.016897                 6.607103   
     82200                            -0.031897                 6.821449   

            cpu__variation_coefficient  cpu__variance  cpu__skewness  ...  \
1    57300                    0.133274      21.247475      -0.009936  ...   
     57600                    0.129521      19.795219      -0.057404  ...   
     57900                    0.130552      20.011681      -0.009214  ...   
     58200                    0.131720      20.328326      -0.008040  ...   
     58500                    0.134915      21.241611      -0.062458  ...   
...                                ...            ...            ...  ...   
1313 81000                    0.214897      35.147240      -0.027028  ...   
     81300                    0.226057      38.106158      -0.078203  ...   
     81600                    0.235499      40.692185      -0.096063  ...   
     81900                    0.246325      43.653805      -0.115089  ...   
     82200                    0.256484      46.532169      -0.127938  ...   

            cpu__fourier_entropy__bins_2  cpu__fourier_entropy__bins_3  \
1    57300                      0.384543                      0.688364   
     57600                      0.384543                      0.688364   
     57900                      0.317937                      0.688364   
     58200                      0.317937                      0.741749   
     58500                      0.317937                      0.787235   
...                                  ...                           ...   
1313 81000                      0.239217                      0.379535   
     81300                      0.239217                      0.379535   
     81600                      0.239217  

In [22]:
extracted_features.isnull().values.any()

True

In [23]:
extracted_features = impute(extracted_features)

In [24]:
extracted_features.shape

(109363, 372)

In [25]:
extracted_features = select_features(extracted_features, y)

In [26]:
extracted_features.shape

(109363, 354)

In [27]:
extracted_features

cpu__sum_values  \
1    57300      2075.200004   
     57600      2061.060004   
     57900      2055.940004   
     58200      2053.760004   
     58500      2049.680004   
...                     ...   
1313 81000      1655.259997   
     81300      1638.439997   
     81600      1625.239998   
     81900      1609.359998   
     82200      1595.759998   

            cpu__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8  \
1    57300                                           0.000000                 
     57600                                           5.340001                 
     57900                                           5.340001                 
     58200                                           5.340001                 
     58500                                           5.340001                 
...                                                       ...                 
1313 81000                                           1.523333                 
     81300                                           1.523333                 
     81600                                           1.523333                 
     81900                                           1.523333                 
     82200                                           1.523333                 

            cpu__fft_coefficient__attr_"real"__coeff_3  \
1    57300                                  -50.406326   
     57600                                  -67.856539   
     57900                                  -69.393889   
     58200                                  -61.091786   
     58500                                  -48.509904   
...                                                ...   
1313 81000                                  -78.930134   
     81300                                  -47.057674   
     81600                                   -6.312826   
     81900                                   33.147187   
     82200                                   72.308154   

            cpu__fft_coefficient__attr_"real"__coeff_15  \
1    57300                                   -20.279996   
     57600                                    -2.599996   
     57900                                    34.419996   
     58200                                     7.719996   
     58500                                   -32.239996   
...                                                 ...   
1313 81000                                    18.359999   
     81300                                     8.780002   
     81600                                    -1.539999   
     81900                                     4.419998   
     82200                                    17.419999   

            cpu__fft_coefficient__attr_"real"__coeff_18  \
1    57300                                    27.047164   
     57600                                   -17.479656   
     57900                                    -0.521976   
     58200                                    23.595911   
     58500                                   -10.620310   
...                                                 ...   
1313 81000                                    16.350396   
     81300                                    -7.175498   
     81600                                     8.983330   
     81900                                    19.730684   
     82200                                    -1.094932   

            cpu__large_standard_deviation__r_0.25  \
1    57300                                    0.0   
     57600                                    0.0   
     57900                                    0.0   
     58200                                    0.0   
     58500                                    0.0   
...                                           ...   
1313 81000                                    0.0   
     81300                                    0.0   
     81600                                    0.0   
     81900                                   

In [28]:
relevance_table = calculate_relevance_table(extracted_features, y)
relevance_table = relevance_table[relevance_table.relevant]
relevance_table.sort_values("p_value", inplace=True)
relevance_table


,feature,type,p_value,relevant
feature,,,,
cpu__sum_values,cpu__sum_values,real,0.000000,True
"cpu__fft_coefficient__attr_""real""__coeff_15","cpu__fft_coefficient__attr_""real""__coeff_15",real,0.000000,True
"cpu__fft_coefficient__attr_""real""__coeff_3","cpu__fft_coefficient__attr_""real""__coeff_3",real,0.000000,True
"cpu__fft_aggregated__aggtype_""centroid""","cpu__fft_aggregated__aggtype_""centroid""",real,0.000000,True
"cpu__fft_coefficient__attr_""imag""__coeff_3","cpu__fft_coefficient__attr_""imag""__coeff_3",real,0.000000,True
...,...,...,...,...
"cpu__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)",cpu__cwt_coefficients__coeff_7__w_2__widths_(2...,real,0.001300,True
"cpu__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",cpu__cwt_coefficients__coeff_0__w_2__widths_(2...,real,0.002629,True
"cpu__fft_coefficient__attr_""imag""__coeff_25","cpu__fft_coefficient__attr_""imag""__coeff_25",real,0.004480,True


In [30]:
extracted_features = extracted_features[relevance_table.reset_index(drop=True).iloc[:n_features, 0].values]

In [31]:
extracted_features

cpu__sum_values  cpu__fft_coefficient__attr_"real"__coeff_15  \
1    57300      2075.200004                                   -20.279996   
     57600      2061.060004                                    -2.599996   
     57900      2055.940004                                    34.419996   
     58200      2053.760004                                     7.719996   
     58500      2049.680004                                   -32.239996   
...                     ...                                          ...   
1313 81000      1655.259997                                    18.359999   
     81300      1638.439997                                     8.780002   
     81600      1625.239998                                    -1.539999   
     81900      1609.359998                                     4.419998   
     82200      1595.759998                                    17.419999   

            cpu__fft_coefficient__attr_"real"__coeff_3  \
1    57300                                  -50.406326   
     57600                                  -67.856539   
     57900                                  -69.393889   
     58200                                  -61.091786   
     58500                                  -48.509904   
...                                                ...   
1313 81000                                  -78.930134   
     81300                                  -47.057674   
     81600                                   -6.312826   
     81900                                   33.147187   
     82200                                   72.308154   

            cpu__fft_aggregated__aggtype_"centroid"  \
1    57300                                 4.528378   
     57600                                 4.230416   
     57900                                 4.240063   
     58200                                 4.237564   
     58500                                 4.258986   
...                                             ...   
1313 81000                                 4.008382   
     81300                                 3.934021   
     81600                                 3.966966   
     81900                                 3.889931   
     82200                                 3.908793   

            cpu__fft_coefficient__attr_"imag"__coeff_3  \
1    57300                                   20.935206   
     57600                                   -0.035348   
     57900                                  -22.584608   
     58200                                  -43.596787   
     58500                                  -61.602198   
...                                                ...   
1313 81000                                 -142.406780   
     81300                                 -165.025315   
     81600                                 -175.569046   
     81900                                 -173.834046   
     82200                                 -159.285589   

            cpu__fft_coefficient__attr_"imag"__coeff_4  \
1    57300                                   18.730944   
     57600                                   19.646036   
     57900                                   15.081658   
     58200                                    7.022839   
     58500                                   -3.909779   
...                                                ...   
1313 81000                                  -48.378235   
     81300                                  -32.745803   
     81600                                  -16.820247   
     81900                                   -4.445296   
     82200                                    3.166669   

            cpu__fft_coefficient__attr_"imag"__coeff_5  \
1    57300                                   13.171162   
     57600                                   25.738698   
     57900                                   28.849570   
     58200                                   23.140224   
     58500                                    9.190473   

In [33]:
extracted_features.reset_index(inplace=True)
extracted_features.rename(columns={
    'level_0': 'machine_id',
    'level_1': 'time',
}, inplace=True)
extracted_features

/tmp/ipykernel_14406/382924147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_features.rename(columns={


,machine_id,time,cpu__sum_values,"cpu__fft_coefficient__attr_""real""__coeff_15","cpu__fft_coefficient__attr_""real""__coeff_3","cpu__fft_aggregated__aggtype_""centroid""","cpu__fft_coefficient__attr_""imag""__coeff_3","cpu__fft_coefficient__attr_""imag""__coeff_4","cpu__fft_coefficient__attr_""imag""__coeff_5","cpu__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8",...,"cpu__fft_aggregated__aggtype_""kurtosis""",cpu__friedrich_coefficients__coeff_3__m_3__r_30,"cpu__linear_trend__attr_""stderr""","cpu__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""var""","cpu__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""min""","cpu__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""var""","cpu__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","cpu__cwt_coefficients__coeff_12__w_5__widths_(2, 5, 10, 20)","cpu__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""min""","cpu__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min"""
0,1,57300,2075.200004,-20.279996,-50.406326,4.528378,20.935206,18.730944,13.171162,0.000000,...,5.054988,227.954523,0.034253,0.198270,-1.0,10.873964,16.214386,23.446304,27.08,-5.44
1,1,57600,2061.060004,-2.599996,-67.856539,4.230416,-0.035348,19.646036,25.738698,5.340001,...,5.455482,265.154965,0.033267,0.699360,-1.0,6.770711,15.355349,23.848532,27.08,-5.44
2,1,57900,2055.940004,34.419996,-69.393889,4.240063,-22.584608,15.081658,28.849570,5.340001,...,5.411539,283.642287,0.033356,0.361474,-1.0,8.277701,17.816524,21.531122,27.08,-5.44
3,1,58200,2053.760004,7.719996,-61.091786,4.237564,-43.596787,7.022839,23.140224,5.340001,...,5.420643,286.216286,0.033223,0.244684,-1.0,10.182519,17.804211,17.479078,24.60,-2.96
4,1,58500,2049.680004,-32.239996,-48.509904,4.258986,-61.602198,-3.909779,9.190473,5.340001,...,5.419782,299.516139,0.033170,0.237401,-1.0,10.584338,15.307700,12.588025,24.60,-2.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109358,1313,81000,1655.259997,18.359999,-78.930134,4.008382,-142.406780,-48.378235,-29.295466,1.523333,...,6.025132,-13.769569,0.043467,-0.027765,-1.0,12.084924,10.532967,-2.446502,18.82,-3.70
109359,1313,81300,1638.439997,8.780002,-47.057674,3.934021,-165.025315,-32.745803,-34.861073,1.523333,...,6.121457,-34.368532,0.044735,-0.014615,-1.0,13.594078,12.231789,-3.758251,18.82,-5.06
109360,1313,81600,1625.239998,-1.539999,-6.312826,3.966966,-175.569046,-16.820247,-37.685683,1.523333,...,6.021644,-18.298198,0.045524,0.159105,-1.0,9.695271,12.513629,-4.993084,18.82,-5.06
109361,1313,81900,1609.359998,4.419998,33.147187,3.889931,-173.834046,-4.445296,-38.352445,1.523333,...,6.365107,-29.119766,0.046439,0.237369,-1.0,6.271000,9.639206,-5.719998,18.82,-5.40


In [34]:
extracted_features.to_csv('extracted_features_60_106.csv', index=None)
y.to_csv('labels_60_106.csv', index=None)